# Parser for Apache access log

In [3]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from apachelogs import LogParser
from user_agents import parse
from crawlerdetect import CrawlerDetect
from tqdm.notebook import tqdm
import time
import os, glob
import json

crawler_detect = CrawlerDetect()
linux_distros = ["Ubuntu", "Debian", "Solaris", "Gentoo", "OpenBSD", "SUSE", "FreeBSD", "Fedora", "Red Hat", "Slackware", "Linux Mint"]

### Parsing each line and adding it to df

I'm going to parse each line and create a Pandas DataFrame to work with the data.
I will flush the rows array to a temp csv every 1M rows to free up memory, and then I will combine these csv in a big one.

In [2]:
#defining the parser params
parser = LogParser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

cols = ["ip", "date", "time", "weekday", "method", "code", "size", "url", "http_v", "bot", "browser", "os", "device", "useragent"]
rows = []
counter = 1 #for csv name
with open("access.log", "r") as fp:
    total = sum(1 for line in fp) #calculate numb of rows for the progressbar
    fp.seek(0) #bring the pointer back to the first line
    for row in tqdm(parser.parse_lines(fp), total=total):
        df_row = {}
        df_row["ip"] = row.remote_host
        datetime = row.request_time
        df_row["date"] = datetime.strftime("%d/%m/%Y")
        df_row["time"] = datetime.strftime("%H:%M:%S")
        df_row["weekday"] = datetime.weekday()
        df_row["code"] = row.final_status
        df_row["size"] = row.bytes_sent
        
        req = row.request_line.split(" ")
        if (len(req)==3):
            df_row["method"] = req[0]
            df_row["url"] = req[1].split('?')[0]
            df_row["http_v"] = req[2]
        else:
            df_row["method"] = None
            df_row["url"] = None
            df_row["http_v"] = None
        if "User-Agent" in row.headers_in and row.headers_in["User-Agent"] is not None:
            df_row["bot"] = crawler_detect.isCrawler(row.headers_in["User-Agent"])
            df_row["device"] = "Bot" #if not true, it will be changed in the next IF
            
            if df_row["bot"] is False:
                ua = parse(row.headers_in["User-Agent"]) 
                df_row["browser"] = ua.browser.family
                
                reqos = ua.os.family
                if reqos in linux_distros:
                    df_row["os"] = "Linux"
                elif reqos in ["Mac OS", "Mac OS X", "macOS"]:
                    df_row["os"] = "macOS"
                else:
                    df_row["os"] = reqos
                
                if ua.is_pc or df_row["os"] == "Windows" or df_row["os"] == "macOS":
                    df_row["device"] = "Desktop"
                elif ua.is_mobile:
                    df_row["device"] = "Smartphone"
                elif ua.is_tablet:
                    df_row["device"] = "Tablet"
                else:
                    df_row["device"] = "Unknown"
                    
            df_row["useragent"] = row.headers_in["User-Agent"]
            
        else:
            df_row["useragent"] = None
        rows.append(df_row)
        
        #if multiple of 1M flush to csv and free up memory
        if (len(rows)%1000000 == 0):
            df = pd.DataFrame(rows, columns=cols)
            df.to_csv("access_"+str(counter)+".csv", sep="\t")
            counter = counter+1
            rows = []

  0%|          | 0/594168 [00:00<?, ?it/s]

In [73]:
#flushing to csv last items left from last flush
df = pd.DataFrame(rows, columns=cols)
df.to_csv("access_"+str(counter)+".csv", sep="\t")
counter = counter+1
rows = []

NameError: name 'rows' is not defined

In [4]:
#importing all csv file to create a single one
files = glob.glob("access_*.csv")
df = pd.concat((pd.read_csv(f, sep="\t") for f in files))

In [5]:
#exporting all csv in a single file
df.to_csv("access.csv", sep="\t")

### Load checkpoint

In [2]:
if "df" not in locals():
    df = pd.read_csv("access.csv", sep="\t")

### Adding Country and City to the dataset for each IP

In [3]:
import geoip2.database

unique_ips = df["ip"].unique() #getting unique ips

def getCityName(names):
    if len(names)==0:
        return ""
    elif "en" in names:
        return names["en"]
    else:
        return names[names.keys()[0]]

rows = []
with geoip2.database.Reader('GeoLite2-City.mmdb') as reader:
    for ip in tqdm(unique_ips, total=len(unique_ips)):
        try:
            info = reader.city(ip)
            rows.append({"lat":info.location.latitude, "lon": info.location.longitude, "city": getCityName(info.city.names), "country":info.country.iso_code})
            df.loc[df["ip"] == ip, "country"] = info.country.iso_code
        except Exception as e:
            print(e)

#creating new df for ips
ips = pd.DataFrame(rows, columns=["lat", "lon", "city", "country"])
del rows

  0%|          | 0/5265 [00:00<?, ?it/s]

The address 127.0.0.1 is not in the database.
The address 104.207.73.126 is not in the database.
The address 104.207.73.83 is not in the database.


In [12]:
#storing country statistics and cities in file
with open("world.json", "w+") as fp:
    countries = df["country"].value_counts().to_dict()
    cities = ips.drop_duplicates().dropna().values.tolist()
    for city in cities:
        if type(countries[city[3]]) is int:
            countries[city[3]] = { "reqs": countries[city[3]], "cities": []}
        countries[city[3]]["cities"].append(city[:-1])
    json.dump({"countries": countries},fp)

In [30]:
#distribution by hour and weekday
def get_dist(country):
    current_df = df if country == "global" else df[df["country"] == country]
    dist = {}
    for wd in range(7):
        dist[wd] = []
        for hh in range(24):
            dist[wd].append(current_df[(current_df["weekday"] == wd) & (current_df["time"].str[0:2] == str(hh).zfill(2))]["time"].shape[0])
    return dist

with open("output/dist.json", "w+") as fp:
    all_dist = {}
    countries = ["global"] + list(df["country"].unique())
    for country in tqdm(countries):
        all_dist[country] = get_dist(country)
    
    json.dump(all_dist,fp)


  0%|          | 0/108 [00:00<?, ?it/s]

In [31]:
#devices, os and browser
def get_devices(country):
    current_df = df if country == "global" else df[df["country"] == country]
    mydevices = current_df[current_df["device"].notnull()]
    devices = []
    for device in mydevices["device"].unique():
        thisdevice = mydevices[mydevices["device"] == device]
        oss = []
        browsers = [] 
        for os in thisdevice[thisdevice["os"].notnull()]["os"].unique():
            oss.append({
                "os": os,
                "value": thisdevice[thisdevice["os"] == os].shape[0]
            })
        for browser in thisdevice[thisdevice["browser"].notnull()]["browser"].unique():
            browsers.append({
                "browser": browser,
                "value": thisdevice[thisdevice["browser"] == browser].shape[0]
            })
        devices.append({
            "device": device,
            "value": thisdevice.shape[0],
            "os": oss,
            "browser": browsers
        })
    return devices

with open("output/devices.json", "w+") as fp:
    all_devices = {}
    for country in tqdm(countries):
        all_devices[country] = get_devices(country)
    json.dump(all_devices, fp)
    

  0%|          | 0/108 [00:00<?, ?it/s]

In [38]:
#http status on requests
def get_status(country):
    current_df = df if country == "global" else df[df["country"] == country]
    return current_df["code"].value_counts().to_dict()

with open("output/status.json", "w+") as fp:
    all_status = {}
    for country in tqdm(countries):
        all_status[country] = get_status(country)
    
    json.dump(all_status,fp)

  0%|          | 0/108 [00:00<?, ?it/s]

In [108]:
#get first time of each user
df2 = df.copy()
df2['date'] = pd.to_datetime(df2['date'], format="%d/%m/%Y")

jgroup = df2.groupby(["ip"], as_index=False)
jgroup = jgroup.agg({
    "date": "min"
})
del df2

In [111]:
#daily global infos
group = df.groupby(["date"], as_index=False)
group = group.agg({
    "ip": "nunique",
    "url": pd.Series.mode,
    "bot": "sum",
    "size": "count"
})
group = group.rename(columns={"size": "requests"})

group_by_country = df.groupby(["date", "country"], as_index=False)
group_by_country = group_by_country.agg({
    "ip": "nunique",
    "url": pd.Series.mode,
    "bot": "sum",
    "size": "count"
})
group_by_country = group_by_country.rename(columns={"size": "requests"})

with open("output/requests.json", "w+") as fp:
    days = {"global":{}}
    for i, row in group.iterrows():
        days["global"][row["date"]] = {
            "reqs": row["requests"],
            "url": row["url"][0],
            "bot": row["bot"],
            "users": row["ip"],
            "newusers": jgroup[jgroup["date"] == pd.to_datetime(row["date"], format="%d/%m/%Y")].shape[0]
        }
    for i, row in group_by_country.iterrows():
        if (row["country"] not in days):
            days[row["country"]] = {}
        days[row["country"]][row["date"]] = {
            "reqs": row["requests"],
            "url": row["url"][0],
            "bot": row["bot"]
        }
    json.dump(days, fp)

        

In [109]:
jgroup["date"].value_counts()

2016-04-12    428
2016-03-22    317
2016-03-24    284
2016-04-05    263
2016-03-23    260
2016-03-25    260
2016-04-04    227
2016-04-11    214
2016-03-21    202
2016-04-13    193
2016-03-28    180
2016-04-08    172
2016-03-31    172
2016-03-29    168
2016-04-02    162
2016-03-27    157
2016-04-01    152
2016-04-19    148
2016-04-03    140
2016-03-30    135
2016-04-07    133
2016-04-15    121
2016-04-06    120
2016-03-26    117
2016-04-10    106
2016-04-20    100
2016-04-21     99
2016-04-14     94
2016-04-09     92
2016-04-26     26
2016-04-16     22
2016-03-16      1
Name: date, dtype: int64